In [12]:
import pandas as pd
from google.colab import files
import io

class dataCleaning():

  def __init__(self):
    # initialize the dataframe

    uploaded = files.upload()
    fileNameAsKey = uploaded.keys()
    filename= list(fileNameAsKey)
    print("**************** File Name ***************************")
    print(f" FileName : {filename[0]} ")

    # print(f"this is uploaded {uploaded.fromkeys.key}")
    df = pd.read_excel(io.BytesIO(uploaded[filename[0]]))
    self.df = df

    self.display_data_info()


    columnsToDrop = self.getColumnNames(self.df)
    df_AfterColumnDrop = self.dropColumns(columnsToDrop)
    df_AfterColumnRename = self.renameColumn(df_AfterColumnDrop)

    status, df_AfterTransform = self.dataTransformation(df_AfterColumnRename)
    if status == True:
      print("************* Df after Transformation  **************")
      print(df_AfterTransform.head(5))
      self.saveToFile(df_AfterTransform)

  def display_data_info(self):
    # Null Summary and info for data frame
    print("**************** Df Null Sum ************************")
    print(self.df.isnull().sum())

    print("**************** Df.info() **************************")
    print(self.df.info())

  def saveToFile(self, df):
    # saving dataframe to file
    print(" ************* Saving to file ********************** ")
    print("...")
    df.to_csv('output.csv', index=False)
    print(" ************* Saved to file :) ******************** ")


  def getColumnNames(self, df):
    # find columns name having at least 10 percent of the total row as null value
    columnNames = df.columns
    columnToDrop = []
    noOfRows = df.shape[0]
    print
    for column in columnNames:
        nullValueSum = sum(df[column].isnull())
        # selecting columns which has at least 10% of null values.
        if nullValueSum > 0.10 * noOfRows:
          #print(f"sum of null values : {nullValueSum}")
          columnToDrop.append(column)

    # Add other column names which are not required in the final data frame
    otherColumnNames = ["Timestamp", "Are you paying rental insurance for your unit monthly?", "What was the total amount you paid initially, covering fees such as application fees, deposit fees, and others?",
                        "Do your apartment require guarantor?", "Since arriving in Wichita, how many times have you relocated to different places?",
                        "What is your preferred mode of transportation when leaving your residence?", "What is the approximate duration of time, in minutes, it takes for you to reach your closest grocery store?",
                        "What are all the amenities that are included in your rent?"]
    for col in otherColumnNames:
      if col not in columnToDrop:
        columnToDrop.append(col)
    return columnToDrop

  def dropColumns(self, columnToDrop):
    # Drop columns with high Null values and columns which are not important
    df_02 = self.df.drop(columns = columnToDrop)
    # Drop columns which are non important
    colNames = df_02.columns
    return df_02

  def dropRowsAndDuplicates(self, df):
    try:
      #drop duplicate rows on the basis of WSU ID column
      df_AfterDuplicate = df.drop_duplicates(subset=['WSU_id'], keep = 'first')
      # drop null/Nan/NA values from the dataframe
      df_AfterNADrop = df_AfterDuplicate.dropna(axis=0, how = 'any')
      # df_Transformed = df_AfterNADrop.reset_index()
      print("**************** Transformed DF Null Sum ************************")
      print(df_AfterNADrop.isnull().sum())
      return True, df_AfterNADrop
    except Exception as e:
      print(f" Error from dropRowsAndDuplicates: {e} ")
      return False



  def renameColumn(self, df):
      #Rename column in dictionary format with old column as key and new column as value
      columnDict= {
              "Are you an international student?" : "If_International_Student",
              "What is your WSU id?" : "WSU_id",
              "Where are you living currently?": "Current_House_Type",
              "How long have you been in Wichita? (in months)" : "Months_Stayed_In_Wichita",
              "What is your apartment name? (street name if housing)" : "House_Name_Or_Address",
              "How many people are staying in your unit?" : "Roommates",
              "What is the size of your unit?" : "Unit_Size",
              "Overall, how satisfied are you with your current living?" : "Overall_Satisfaction",
              "On a scale of responsiveness, how efficient is the management of your apartment complex? (in maintenance services etc.,)" : "Management_Efficiency",
              "What are your thoughts on the level of safety in the vicinity of your residence?" : "Safety_Level",
              "What is the total monthly rent of your unit?" : "Monthly_Rent",
              "What is the approximate total amount of monthly bills (electricity, parking, etc.,) not included in your rent?" : "Other_Bills",
              "Is your apartment pet friendly?" : "If_Pet_Friendly",
              "What is the approximate duration of time it takes for you to reach your preferred nearby restaurant?" : "Time_Taken_to_reach_Preferred_restaurant",
              "What type of laundry amenities are available in your accommodation?" : "Laundry_Condition"
      }
      try:
        df_AfterColumnRename = df.rename(columns = columnDict)
        return df_AfterColumnRename
      except Exception as e:
        print(f"Error in Column Rename: {e}")
        return False

  def dataTransformation(self, df):
      # change suitable data type for all columns
      try:
        df['If_International_Student'] = (df['If_International_Student'].apply(lambda x : str(x) if isinstance(x, str) else pd.NA )).astype('string')
        df['WSU_id'] = (df['WSU_id'].apply(lambda x : str(x).upper() if isinstance(x, str) else pd.NA )).astype('string')
        df['Current_House_Type'] = (df['Current_House_Type'].apply(lambda x : str(x) if isinstance(x, str) else pd.NA )).astype('string')
        df['Months_Stayed_In_Wichita'] = (df['Months_Stayed_In_Wichita'].apply(lambda x : int(x) if isinstance(x, int) else pd.NA )).astype('Int64')
        df['House_Name_Or_Address'] = (df['House_Name_Or_Address'].apply(lambda x : str(x) if isinstance(x, str) else pd.NA )).astype('string')
        df['Roommates'] = (df['Roommates'].apply(lambda x : int(x) if isinstance(x, int) else pd.NA )).astype('Int64')
        df['Unit_Size'] = (df['Unit_Size'].apply(lambda x : str(x) if isinstance(x, str) else pd.NA)).astype('string')
        df['Monthly_Rent'] = (df['Monthly_Rent'].apply(lambda x : int(x) if isinstance(x, int) else pd.NA )).astype('Int64')
        df['Other_Bills'] = (df['Other_Bills'].apply(lambda x : int(x) if isinstance(x, int) else pd.NA )).astype('Int64')
        df['If_Pet_Friendly'] = (df['If_Pet_Friendly'].apply(lambda x : str(x) if isinstance(x, str) else pd.NA )).astype('string')
        df['Time_Taken_to_reach_Preferred_restaurant'] = (df['Time_Taken_to_reach_Preferred_restaurant'].apply(lambda x : int(x) if isinstance(x, int) else pd.NA )).astype('Int64')
        df['Laundry_Condition'] = (df['Laundry_Condition'].apply(lambda x : str(x) if isinstance(x, str) else pd.NA )).astype('string')

        print("**************** Duplicates in WSU Id after type cast ********************")
        print(df['WSU_id'].duplicated().sum())

        status , df_transform = self.dropRowsAndDuplicates(df)
        if status == True:
          return True, df_transform
      except Exception as e:
        print(f" Error in Data Transformation : {e} ")
        return False

dc = dataCleaning()


Saving questionnaire_responses.xlsx to questionnaire_responses (7).xlsx
**************** File Name ***************************
 FileName : questionnaire_responses (7).xlsx 
**************** Df Null Sum ************************
Timestamp                                                                                                                      0
Are you an international student?                                                                                              0
This questionnaire helps the new international students coming to WSU. Do you want to fill this form?                       1172
What is your WSU id?                                                                                                          14
Where are you living currently?                                                                                                8
How long have you been in Wichita? (in months)                                                                                 8